In [2]:
import os
import pandas as pd
import numpy as np
from openpyxl import *
from openpyxl.styles import Font, Border, Side, Alignment, colors
import re
import sys
sys.path.append("../scripts/")

from process import bidExcelProcessing, bidExcelFormat
from extract import filterDownloadExcel, relatedSheetProcessing

cpath = "/Users/junyelu/Desktop/cost.xlsx"
ppath = "/Users/junyelu/Desktop/Parameters.xlsx"


def migration(iPath, outpath):
    '''
    Mapping completed table onto worksheets ready for submission
    ---
    input: confirmed Excelsheets
    output: Excelsheets ready for submssion
    ---
    '''
    #Input and process all files, locate relevant sheets
    inputFile = pd.read_excel(ipath, sheet_name=None, converters={'网省采购申请号': int})
    table = pd.read_excel(outpath, skiprows=0, usecols='A:S', converters={'网省采购申请行号': int})
    table = table.drop('未含税单价(万)', axis=1)
    if len(table.loc[0,'分包名称']) == 2:
        table.loc[:, '分包名称'] = table.loc[:, '分包名称'].apply(lambda x: x.replace('包', '包0') if len(x) == 2 else x)
    for _, sheet in inputFile.items():
        if sheet.loc[0,'网省采购申请号'] in list(table['网省采购申请行号']):
            inputSheet = sheet
            break
    
    #Migrate data
    temp = inputSheet.loc[:,['包名称', '网省采购申请号', '物资名称', '未含税单价']]
    temp = temp.rename(columns={'包名称': '分包名称', '网省采购申请号': '网省采购申请行号', '物资名称': '物料名称', '未含税单价': '未含税单价(万)'})
    try:
        table = pd.merge(table, temp, on=['分包名称', '物料名称','网省采购申请行号'])
    except:
        return f"{outpath.split('/')[-1].split('.')[0]} 存在数据问题"
    
    #Followup with computations
    table['税率（%）'] = 13
    table['含税单价(万)'] = table['未含税单价(万)'] * (1 + table['税率（%）']/100) 
    table['未含税合价(万)'] = table['未含税单价(万)'] * table['数量']
    table['含税合价(万)'] = table['含税单价(万)'] * table['数量']

    #Check if two totals is the same
    if not np.isclose(sheet.tail(1)['含税总价'].values[0], table.sum()['含税合价(万)'], rtol=0, atol=1e-10):
        return f"{outpath.split('/')[-1].split('.')[0]} 与表格总价差异较大"

    new_cols = ['分包编号', '分包名称', '轮次', '附件上传状态', '物料编码', '物料名称', '技术规范书ID', '网省采购申请行号',
       '项目单位', '单位', '扩展描述', '包限价(万)', '行限价(万)', '数量', '未含税单价(万)', '税率（%）', '含税单价(万)',
       '未含税合价(万)', '含税合价(万)']
    finalTable = table.reindex(columns = new_cols)

    #Safeguard against price ceilings
    if any(table['行限价(万)'].astype(str).str.contains('\d', regex=True)):
        if any(table['行限价(万)'] < table['含税合价(万)']):
            return f"{outpath.split('/')[-1].split('.')[0]} 超过行限价"

    with pd.ExcelWriter(outpath, mode='a', engine='openpyxl', if_sheet_exists='new') as writer:
        finalTable.to_excel(writer, sheet_name='报价方式-单价', index=False, startrow=1, startcol=1)
 

In [ ]:
opath = '/Users/junyelu/Desktop/货物清单_国网上海市电力公司2022年新增用户接入工程物资协议库存招标采购20221122_121646_179.xlsx'
filterDownloadExcel(opath)

In [3]:
rpath = '/Users/junyelu/Desktop/相关清单/黑龙江省相关清单.xlsx'
relatedSheetProcessing(rpath, ppath)

In [ ]:
loc = "山东"
fpath = "/Users/junyelu/Desktop/相关清单/"+loc+"省相关清单.xlsx"
outpath = "/Users/junyelu/Desktop/"+loc+".xlsx"

bidExcelProcessing(fpath, cpath, ppath, outpath)
bidExcelFormat(outpath)

In [ ]:
#Path migration
ipath = '/Users/junyelu/Desktop/宁夏.xlsx'
outpath = "/Users/junyelu/Desktop/宁夏架空绝缘导线（一）.xlsx"
migration(ipath, outpath)

In [ ]:
#Parameters reassessing
import os

import pandas as pd
sourceFolder = '/Users/junyelu/Desktop/Files Compost/2021 Work/Bidding/SGCC1'
pd.set_option('display.max_rows', 999)

fileAddress = []
for root, dirs, files in os.walk(sourceFolder):
    if '江苏' in root and '开标' in root:
        for file in files:
            if '金具' not in file:
                fileAddress.append(root+'/'+file)

tmp = pd.read_excel(fileAddress[1], sheet_name=None)
for key, sheet in tmp.items():
    print(sheet.iloc[5, -2] + sheet.iloc[3, -2])


In [ ]:
import os
import pandas as pd
sourceFolder = '/Users/junyelu/Desktop/Files Compost/Work/2022 Bidding/SGCC1'
pd.set_option('display.max_rows', 999)

folders_path = []
for file in os.listdir(sourceFolder):
    if '开标' in file:
        folders_path.append(sourceFolder+"/"+file)

concat_files = []
required_columns = ['分标编号','分标名称','项目单位','分包名称','投标人名称','投标价格']
for folder_path in folders_path:
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx') and '金具' not in file:
            print(f'Processing {folder_path, file}')
            concat_files.append(pd.concat(pd.read_excel(folder_path+"/"+file, sheet_name=None, usecols=required_columns), ignore_index=True))
concat_files

In [ ]:
prov_202102_pd = pd.DataFrame.from_dict(prov_202102, orient="index", columns=['Occurence2102'])

In [ ]:
prov_202101_pd = pd.DataFrame.from_dict(prov_202101, orient="index", columns=['Occurence2101'])

In [ ]:
prov_202201_pd = pd.DataFrame.from_dict(prov_202201, orient="index", columns=['Occurence2201'])

In [ ]:
temp = prov_202101_pd.join(prov_202102_pd, how="outer")

In [ ]:
pd.concat(concat_files, ignore_index=True).to_excel('统计.xlsx')

In [ ]:
import pandas
pandas.__version__

In [ ]:
import openpyxl
openpyxl.__version__